In [ ]:
import numpy as np
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
from sklearn.decomposition import PCA
import math
from scipy.spatial import cKDTree
from numbers import Number
import torch.nn.functional as F
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 
from sklearn.mixture import GaussianMixture
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from sklearn import mixture
import torch.optim as optim
from scipy.optimize import linear_sum_assignment
import scanpy as sc
from anndata import AnnData
import torch
import math
from collections import Counter
from sklearn.neighbors import NearestNeighbors
from sklearn.covariance import EllipticEnvelope
from scipy.stats import mode
from torch_geometric.data import Data
from sklearn.neighbors import LocalOutlierFactor
import umap
from torch_geometric.nn import GCNConv
import matplotlib
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.mixture import BayesianGaussianMixture
import torch.nn as nn
from sklearn.metrics.cluster import adjusted_rand_score
import torch
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cmap = {
    "Skin-SHARE": {
    'Dermal Fibroblast': (0.5, 0.5, 0.5),          # Gray for fibroblast cells
    'Dermal Papilla': (0.52, 0.52, 0.52),         # Slightly different gray
    'TAC-1': (0.9, 0.8, 0.1),                     # Yellow for basal keratinocyte progenitors
    'Mix': (0.6, 0.6, 0.6),                       # Light gray for mixed cells
    'IRS': (0.6, 0.1, 0.5),                       # Purple for inner root sheath
    'Basal': (0.9, 0.82, 0.12),                   # Slightly different yellow
    'K6+ Bulge Companion Layer': (0.58, 0.1, 0.48),  # Slightly different purple
    'Medulla': (0.62, 0.12, 0.52),                # Another shade of purple
    'alowCD34+ bulge': (0.64, 0.14, 0.54),        # Another shade of purple
    'Isthmus': (0.66, 0.16, 0.56),                # Another shade of purple
    'ORS': (0.68, 0.18, 0.58),                    # Another shade of purple
    'Infundibulum': (0.7, 0.2, 0.6),              # Another shade of purple
    'Spinous': (0.92, 0.84, 0.14),                # Another shade of yellow
    'ahighCD34+ bulge': (0.72, 0.22, 0.62),       # Another shade of purple
    'TAC-2': (0.94, 0.86, 0.16),                  # Another shade of yellow
    'Macrophage DC': (0.2, 0.6, 0.3),             # Green for immune cells
    'Endothelial': (0.3, 0.7, 0.4),               # Slightly different green
    'Dermal Sheath': (0.54, 0.54, 0.54),          # Another gray shade
    'Sebaceous Gland': (0.62, 0.62, 0.62),        # Another light gray
    'Granular': (0.96, 0.88, 0.18),               # Another shade of yellow
    'Hair Shaft-cuticle.cortex': (0.74, 0.24, 0.64), # Another shade of purple
    'Schwann Cell': (0.64, 0.64, 0.64),           # Another light gray
    'Melanocyte': (0.66, 0.66, 0.66)              # Another light gray
},
    "BM-CITE": {
    'CD14 Mono': (0.1, 0.4, 0.6),        # Dark blue
    'CD16 Mono': (0.1, 0.3, 0.5),        # Lighter blue
    'CD4 Memory': (0.9, 0.4, 0.0),       # Dark orange
    'CD4 Naive': (1.0, 0.6, 0.2),        # Light orange
    'CD8 Effector': (0, 1, 0),     # Dark green
    'CD8 Memory': (0.2, 0.7, 0.2),       # Medium green
    'CD8 Naive': (0.7, 1.0, 0.7),        # Light green
    'GMP': (0.5, 0.3, 0.7),              # Purple
    'HSC': (0.0, 0.5, 0.7),              # Cyan
    'LMPP': (0.0, 0.4, 0.6),             # Darker Cyan
    'MAIT': (0.6, 0.2, 0.2),             # Maroon
    'Memory B': (0.9, 0.3, 0.3),      # Dark red
    'NK': (0.4, 0.3, 0.3),           # Pink
    'Naive B': (1, 0.0, 0.0),          # Light purple
    'Plasmablast': (0.6, 0.6, 0.1),      # Olive
    'Prog_B': (0.5, 0.3, 0.1),             # Brown
    'Prog_DC': (0.6, 0.4, 0.2),          # Darker brown
    'Prog_Mk': (0.7, 0.5, 0.3),          # Even darker brown
    'Prog_RBC': (0.8, 0.6, 0.4),         # Light brown
    'Treg': (0.9, 0.5, 0.3),             # Coral
    'cDC2': (0.3, 0.3, 0.9),             # Indigo
    'gdT': (0.2, 0.5, 0.3),              # Sea green
    'pDC': (0.1, 0.6, 0.6)},
    
    "PBMC-TEA": {
    'T.CD4.Memory': (0.9, 0.4, 0.0),          # Dark orange for CD4 Memory T cells
    'Mono.CD14': (0.1, 0.4, 0.6),             # Dark blue for CD14+ Monocytes
    'B.Naive': (1.0, 0.6, 0.2),               # Light orange for Naive B cells
    'T.CD8.Effector': (0, 1, 0),              # Bright green for CD8 Effector T cells
    'T.CD4.Naive': (1.0, 0.65, 0.25),         # Slightly lighter orange for CD4 Naive T cells
    'B.Activated': (0.95, 0.55, 0.15),        # Darker orange for Activated B cells
    'T.CD8.Naive': (0.7, 1.0, 0.7),           # Light green for CD8 Naive T cells
    'DC.Myeloid': (0.3, 0.3, 0.9),            # Indigo for Myeloid Dendritic Cells
    'NK': (0.4, 0.3, 0.3),                    # Pink for Natural Killer cells
    'Platelets': (0.6, 0.4, 0.4),             # Dusty pink for Platelets
    'Mono.CD16': (0.1, 0.35, 0.55),           # Lighter blue for CD16+ Monocytes
    'T.DoubleNegative': (0.5, 0.5, 0.1)},
    
    "PBMC-Multiome": {
    'CD4 Naive': (0.9, 0.8, 0.4),         # Lighter orange, indicative of naive state
    'CD4 Memory': (0.9, 0.6, 0.2),        # Darker orange, indicative of memory state
    'CD8 Naive': (0.7, 0.9, 0.4),         # Lighter green, for naive CD8
    'CD8 effector': (0, 0.8, 0),          # Dark green, for effector CD8
    'CD16+ Monocytes': (0.1, 0.5, 0.6),   # Deep teal, distinguishing them from CD14+ Monocytes
    'CD14+ Monocytes': (0.1, 0.4, 0.6),   # Dark blue, common for monocytes
    'B cell progenitor': (1, 0.5, 0.75),  # Light pink, indicating an early stage in B cell development
    'pre-B cell': (1, 0.4, 0.7),          # Slightly darker pink, a later stage of B cell progenitor
    'Dendritic cell': (0.5, 0.5, 0.9),    # Bright purple, unique among APCs
    'Double negative T cell': (0.6, 0.3, 0.5),  # Purplish-pink, unique due to their DN nature
    'pDC': (0.4, 0.6, 0.9),               # Lighter purple, indicating a specialized dendritic cell type
    'NK cell': (0.9, 0.3, 0.3),           # Red, vibrant and distinct for natural killer cells
    'Platelets': (0.8, 0.6, 0.4)          # Muted brown, indicative of their different nature
},
    
    "PBMC-DOGMA": {
    'CD4 Naive': (0.9, 0.8, 0.4),
    'CD4 TCM': (0.9, 0.75, 0.3),          # Central memory, close to naive but slightly darker
    'gdT': (0.4, 0.5, 0.2),                # Unique olive green, distinct from other T cells
    'CD8 TEM': (0.2, 0.7, 0.3),            # Effector memory, darker green
    'B naive': (0.6, 0.8, 0.9),            # Light blue for naive B cells
    'CD8 Naive': (0.6, 0.9, 0.5),          # Lighter green for naive CD8
    'Treg': (0.9, 0.5, 0.6),               # Soft pink for regulatory T cells
    'CD14 Mono': (0.1, 0.4, 0.6),          # Standard dark blue for monocytes
    'Eryth': (0.8, 0.3, 0.3),              # Reddish for erythrocytes (even if not immune cells, assuming hematopoietic context)
    'NK': (0.7, 0.2, 0.2),                 # Dark red for natural killer cells
    'B intermediate': (0.55, 0.75, 0.95),  # Intermediate blue for intermediate B cells
    'MAIT': (0.5, 0.4, 0.2),               # Muddy gold for mucosal-associated invariant T cells
    'ASDC': (0.5, 0.5, 0.8),               # Light purple for activated stromal dendritic cells
    'dnT': (0.45, 0.6, 0.25),              # Dark olive for double negative T
    'CD4 TEM': (0.85, 0.7, 0.35),          # Effector memory, near CD4 Naive but slightly darker
    'CD8 TCM': (0.5, 0.85, 0.4),           # Central memory, close to CD8 Naive but distinct
    'CD16 Mono': (0.15, 0.45, 0.65),       # Slightly lighter blue than CD14+ Monocytes
    'ILC': (0.3, 0.6, 0.3),                # Intermediate green for innate lymphoid cells
    'B memory': (0.5, 0.7, 0.9),           # Deeper blue for memory B cells
    'cDC2': (0.4, 0.4, 0.9),               # Indigo for conventional dendritic cell type 2
    'pDC': (0.35, 0.35, 0.85),             # Lighter indigo for plasmacytoid dendritic cells
    'NK_CD56bright': (0.75, 0.25, 0.25),   # Bright red for CD56 bright NK cells
    'CD4 CTL': (0.88, 0.65, 0.4),          # Cytotoxic T lymphocytes, close to CD4 colors but distinct
    'HSPC': (0.9, 0.4, 0.4),               # Hematopoietic stem and progenitor cells, distinct red
    'Platelet': (0.8, 0.6, 0.4),           # Muted brown for platelets
    'Plasmablast': (0.6, 0.6, 0.1),        # Olive for plasmablasts
    'CD4 Proliferating': (0.93, 0.83, 0.45), # Very light orange, indicating active proliferation
    'CD4': (0.9, 0.8, 0.4),
    'CD8': (0.6, 0.9, 0.5),
    },
    
    "LUNG-CITE": {
    'T.CD.EM': (0.2, 0.7, 0.3),           # Dark green for T CD Effector Memory
    'mDC.Lung': (0.5, 0.5, 0.9),          # Light purple for myeloid DC in lung
    'NK': (0.7, 0.2, 0.2),                # Dark red for natural killer cells
    'Mono': (0.1, 0.4, 0.6),              # Dark blue for monocytes
    'T.CD4': (0.9, 0.6, 0.2),             # Orange for CD4 T cells
    'B.Plasma': (0.8, 0.5, 0.9),          # Light violet for plasma B cells
    'T.CD8': (0.6, 0.8, 0.2),             # Olive green for CD8 T cells
    'T.CD8.CM': (0.55, 0.75, 0.25),       # Slightly different olive for CD8 Central Memory
    'B.Blood': (0.6, 0.6, 0.9),           # Soft blue for blood B cells
    'mDC.Blood': (0.45, 0.45, 0.85),      # Slightly different light purple for myeloid DC in blood
    'Macro': (0.9, 0.3, 0.3),             # Coral for macrophages
    'Fibro': (0.6, 0.6, 0.6),             # Grey for fibroblasts
    'T.CD8.EM': (0.3, 0.7, 0.4),          # Different shade of green for CD8 Effector Memory
    'T.Treg': (0.9, 0.5, 0.6),            # Soft pink for T regulatory cells
    'B.Lung': (0.65, 0.65, 0.95),         # Different soft blue for lung B cells
    'Mono.Int': (0.15, 0.45, 0.65),       # Slightly different blue for intermediate monocytes
    'T.Prolif': (0.7, 0.85, 0.4),         # Bright olive for proliferating T cells
    'pDC': (0.35, 0.35, 0.9),             # Light indigo for plasmacytoid dendritic cells
    'Epithelia': (0.8, 0.4, 0.4)          # Muted red for epithelial cells
}
}

additional = {
    -1: (0.34, 0.92, 0.29),
    0: (0.74, 0.54, 0.41),
    1: (0.13, 0.57, 0.76),
    2: (0.88, 0.35, 0.47),
    3: (0.69, 0.16, 0.82),
    4: (0.46, 0.87, 0.34),
    5: (0.25, 0.56, 0.96),
    6: (0.84, 0.38, 0.94),
    7: (0.78, 0.61, 0.52),
    8: (0.17, 0.93, 0.63),
    9: (0.33, 0.48, 0.75),
    10: (0.54, 0.84, 0.12),
    11: (0.69, 0.18, 0.87),
    12: (0.37, 0.91, 0.24),
    13: (0.63, 0.26, 0.56),
    14: (0.71, 0.82, 0.48),
    15: (0.29, 0.72, 0.59),
    16: (0.85, 0.36, 0.97),
    17: (0.19, 0.79, 0.88),
    18: (0.61, 0.47, 0.35),
    19: (0.76, 0.23, 0.67),
    20: (0.91, 0.54, 0.11),
    21: (0.42, 0.78, 0.82),
    22: (0.56, 0.69, 0.37)
}

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print ('Using ', device)

In [ ]:
#BM-CITE  LUNG-CITE  PBMC-DOGMA  PBMC-Multiome  PBMC-TEA 
dataset = 'BM-CITE'
data_dir = '/Users/francescoceccarelli/Library/Mobile Documents/com~apple~CloudDocs/PhD/MultiOmics/preprocessed_data'
files = [f for f in listdir(join(data_dir, dataset)) if isfile(join(data_dir, dataset, f))]

color_map = cmap[dataset]
for k, v in additional.items():
    color_map[k] = v

for file in files:
    if ('EXPR' in file):
        exp = pd.read_csv(join(data_dir, dataset, file))
        exp = exp.T
        exp.columns = exp.iloc[0]
        exp = exp.drop(exp.index[0])

    if ('META' in file): 
        meta = pd.read_csv(join(data_dir, dataset, file))
        meta.index = meta.iloc[:,0]
        meta = meta.drop(meta.columns[0], axis=1)

assert (np.all(meta.index == exp.index))
print ('Expression: ', exp.shape)
print ('Meta: ', meta.shape)

In [ ]:
np.unique(meta['celltype'])

In [ ]:
# Filter out low count cells
# Change labels to be more consistent
min_count = 100
exp['celltype'] = meta['celltype']

if (dataset == 'BM-CITE'):
     exp["celltype"] = exp["celltype"].replace(['CD8 Effector_1', 'CD8 Effector_2'], 'CD8 Effector')
     exp["celltype"] = exp["celltype"].replace(['CD8 Memory_1', 'CD8 Memory_2'], 'CD8 Memory')
     exp["celltype"] = exp["celltype"].replace(['Prog_B 1', 'Prog_B 2'], 'Prog_B')
     exp["celltype"] = exp["celltype"].replace(['CD56 bright NK'], 'NK')
     
if (dataset == 'LUNG-CITE'):
     exp["celltype"] = exp["celltype"].replace(['B.Plasma.1', 'B.Plasma.2'], 'B.Plasma')
     exp["celltype"] = exp["celltype"].replace(['Mono.1', 'Mono.2', 'Mono.3'], 'Mono')

if (dataset == 'PBMC-DOGMA'):
      exp["celltype"] = exp["celltype"].replace(['CD4 Naive', 'CD4 TCM', 'CD4 TEM', ], 'CD4')
      exp["celltype"] = exp["celltype"].replace(['CD8 Naive', 'CD8 TEM', 'CD8 TCM'], 'CD8')

value_counts = exp['celltype'].value_counts()
values_to_remove = value_counts[value_counts < min_count].index
exp = exp[~exp['celltype'].isin(values_to_remove)]

In [ ]:
unique, counts = np.unique(exp["celltype"], return_counts=True)
celltype_counts = exp['celltype'].value_counts()

#Look at the distribution of celltypes
print ('Total number of cell types: ', len(celltype_counts))
print (celltype_counts)

In [ ]:
def gini_index(class_counts):
    """
    Calculate the Gini index using numpy for a list of class counts.

    Args:
    class_counts (list or numpy array): A list or numpy array where each element represents the number of instances in a class.

    Returns:
    float: The Gini index, where 0 represents perfect equality (all classes have the same number of instances),
           and 1 represents maximal inequality (all instances are in one class).
    """
    # Convert class_counts to a numpy array for vectorized operations if it's not already one
    class_counts = np.array(class_counts)
    # Total number of instances
    total_instances = np.sum(class_counts)
    if total_instances == 0:
        return 0  # To handle the case where the dataset is empty and avoid division by zero

    # Calculate the probability of each class (p_i)
    class_probabilities = class_counts / total_instances
    # Calculate the Gini index using the formula: G = 1 - sum(p_i^2)
    gini = 1 - np.sum(class_probabilities ** 2)
    return gini

class_counts = celltype_counts.to_numpy()
print(f"Gini Index: {gini_index(class_counts):.3f}")

In [ ]:
#Split the data into known and unknown classes
spliting_method = 'most_common'
if (spliting_method == 'most_common'):

    known_celltypes = celltype_counts.nlargest(math.ceil(len(unique) / 2)).index
    unknown_celltypes = celltype_counts.nsmallest(math.floor(len(unique) / 2)).index

    known_genes = exp[exp['celltype'].isin(known_celltypes)]
    unknown_genes = exp[exp['celltype'].isin(unknown_celltypes)]

if (spliting_method == 'order'):
    split_point = len(unique) // 2 if len(unique) % 2 == 0 else (len(unique) // 2) + 1

    known_celltypes = unique[:split_point]
    unknown_celltypes = unique[split_point:]

    known_genes = exp[exp['celltype'].isin(known_celltypes)]
    unknown_genes = exp[exp['celltype'].isin(unknown_celltypes)]

if (spliting_method == 'random'):
    known_celltypes = np.random.choice(unique, math.ceil(len(unique) / 2), replace=False)
    unknown_celltypes = np.setdiff1d(unique, known_celltypes)

    known_genes = exp[exp['celltype'].isin(known_celltypes)]
    unknown_genes = exp[exp['celltype'].isin(unknown_celltypes)]
    
print ('Known classes: %d' % len(known_celltypes))
print ('Unkown classes: %d' % len(unknown_celltypes))

print (known_genes.shape)
print (unknown_genes.shape)

In [ ]:
move_percentage = 0.3

#Sample the known cells and move them to the unknown set
group_sizes = known_genes.groupby('celltype').size()
total_size = group_sizes.sum()
weights = group_sizes / total_size
row_weights = known_genes['celltype'].map(weights)

# Sample rows with the computed weights
num_samples = int(len(known_genes) * move_percentage) 
sampled_df = known_genes.sample(n=num_samples, weights=row_weights, replace=False)
unknown_genes = pd.concat([unknown_genes, sampled_df])
known_genes = known_genes.drop(sampled_df.index)
sampled_indices = sampled_df.index

print ('Known cells', known_genes.shape)
print ('Unkown cells', unknown_genes.shape)

In [ ]:
celltype_counts = known_genes['celltype'].value_counts()
print ('Distribution of known cells')
print (celltype_counts)

In [ ]:
# Map the cell types to integers
# Assign integer values to known cell types in [0,..,K-1]
# Assign integer values to unknown cell types in [K,..,N-1]

known_types = known_genes['celltype'].unique()
known_mapping = {cell_type: idx for idx, cell_type in enumerate(known_types)}
known_genes['celltype'] = known_genes['celltype'].map(known_mapping)

all_mapping = known_mapping.copy()
next_int = len(known_mapping)

# Assign integer values to unknown cell types
for cell_type in unknown_genes['celltype'].unique():
    if cell_type not in all_mapping:
        all_mapping[cell_type] = next_int
        next_int += 1

# Apply the mapping to all_df
unknown_genes['celltype'] = unknown_genes['celltype'].map(all_mapping)

known_celltypes_names = known_celltypes
unknown_celltypes_names = unknown_celltypes

known_celltypes = pd.Series(known_celltypes).map(known_mapping)
unknown_celltypes = pd.Series(unknown_celltypes).map(all_mapping)

inv_known_mapping = {v: k for k, v in known_mapping.items()}
inv_all_mapping = {v: k for k, v in all_mapping.items()}

In [ ]:
# Keep track of the unknown cells for differential analysis
unknown_cell_names = np.array(unknown_genes.index)

celltypes_k = known_genes['celltype']
known_genes = known_genes.drop('celltype', axis=1)
known_genes_size = known_genes.shape[0]

celltypes_u = unknown_genes['celltype']
unknown_genes = unknown_genes.drop('celltype', axis=1)
unknown_genes_size = unknown_genes.shape[0]

In [ ]:
#PCA 
#Reduce dimension of the expression matrix
reduction = 'PCA'
pcs_n = 64
if (reduction == 'PCA'):
    print ('Computing PCA with %d components...' %pcs_n)
    pca = PCA(n_components = pcs_n)
    #known_genes = pd.DataFrame(pca.fit_transform(known_genes))
    #unknown_genes = pd.DataFrame(pca.transform(unknown_genes))
    all_genes = pd.concat([known_genes, unknown_genes])
    all_genes = pd.DataFrame(pca.fit_transform(all_genes))
    known_genes = all_genes.iloc[:known_genes_size]
    unknown_genes = all_genes.iloc[known_genes_size:]

known_genes['celltype'] = celltypes_k.tolist()
unknown_genes['celltype'] = celltypes_u.tolist()

print ('Known cells: ', known_genes.shape)
print ('Unknown cells: ', unknown_genes.shape)

In [ ]:
#Define all model inputs 
known_genes = known_genes.to_numpy(dtype=np.float16)
unknown_genes = unknown_genes.to_numpy(dtype=np.float16)

x_labeled = torch.from_numpy(known_genes[:,:-1]).float()
y_labeled = torch.from_numpy(known_genes[:,-1])
y_labeled = y_labeled.type(torch.LongTensor)

x_unlabeled = torch.from_numpy(unknown_genes[:,:-1]).float()
y_unlabeled = torch.from_numpy(unknown_genes[:,-1])
y_unlabeled = y_unlabeled.type(torch.LongTensor)

print (x_labeled.shape)
print (y_labeled.shape)

print (x_unlabeled.shape)
print (y_unlabeled.shape)

In [ ]:
class MaskedAutoEncoder(torch.nn.Module):
    def __init__(self, input_size, hidden_size=[32, 32], dropout=0.2, mask_prob=0.2):
        super().__init__()
        self.input_size = input_size
        self. mask_prob = torch.tensor(mask_prob)

        self.encoder, self.mask_predictor, self.decoder = self.contrsuct_model(input_size, hidden_size, dropout)
        
    def contrsuct_model(self, input_size, hidden_size, dropout):
        
        # Dynamically create the encoder layers
        layers = [nn.Dropout(p=dropout)]
        input_size = input_size
                
        if isinstance(hidden_size, Number):
            hidden_size = [hidden_size]
            
        hidden_size.insert(0, input_size)
        
        for idx in range(len(hidden_size)-2):
            layers.extend([
                nn.Linear(hidden_size[idx], hidden_size[idx+1]),
                nn.LayerNorm(hidden_size[idx+1]),
                nn.Mish(inplace=True)
            ])
        
        # Last layer of the encoder without activation
        layers.append(nn.Linear(hidden_size[-2], hidden_size[-1]))

        encoder = nn.Sequential(*layers)

        mask_predictor = nn.Linear(hidden_size[-1], input_size)
        decoder = nn.Linear(in_features=hidden_size[-1] + input_size, out_features=input_size)
        
        return encoder, mask_predictor, decoder

    def forward(self, X):
        
        if self.training:
            corrupted_X, true_mask = self.mask_input(X)
        else:
            corrupted_X = X
            true_mask = torch.ones_like(X)
        
        encoded = self.encoder(corrupted_X)
        predicted_mask = self.mask_predictor(encoded)
        masked_encoded = torch.cat([encoded, true_mask * corrupted_X], dim=-1)
        reconstruction = self.decoder(masked_encoded)
        
        return [encoded, reconstruction]
    
    
    def mask_input(self, X):
        
        should_swap = torch.bernoulli(self.mask_prob.to( X.device) * torch.ones((X.shape)).to(X.device))
        corrupted_X = torch.where(should_swap == 1, X[torch.randperm(X.shape[0])], X)
        masked = (corrupted_X != X).float()
        return corrupted_X, masked


class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded
    
class Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Predictor, self).__init__()
        self.predictor = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
            nn.Softmax(dim=1))

    def forward(self, x):
        return self.predictor(x)
    
class MultiObjectiveLoss(nn.Module):
    def __init__(self):
        super(MultiObjectiveLoss, self).__init__()
        self.weights = nn.Parameter(torch.rand(2)) 
    
    def forward(self, loss1, loss2):
        # Compute the weighted sum of the loss terms
        weights = F.softmax(self.weights, dim=0)
        weighted_loss = self.weights[0] * loss1 + self.weights[1] * loss2
        return weighted_loss

In [ ]:
x_labeled = x_labeled.to(device)
y_labeled = y_labeled.to(device)
x_unlabeled = x_unlabeled.to(device)
y_unlabeled = y_unlabeled.to(device)

In [ ]:
input_size = x_labeled.shape[1]
hidden_size = 32
masked = False

#all data for unsupervised autoencoder loss
input_data = torch.cat((x_labeled, x_unlabeled), dim=0).to(device)

# Instantiate the autoencoder, predictor & pseudo_predictor
if masked:
    autoencoder = MaskedAutoEncoder(input_size).float().to(device)
    print ("Using Masked Autoencoder...")
else:
    autoencoder = Autoencoder(input_size, hidden_size).float().to(device)

predictor = Predictor(input_size, hidden_size, len(known_celltypes)).float().to(device)

#losses 
criterion_mse = nn.MSELoss() #autoencoder
criterion_CE = nn.CrossEntropyLoss() #predictor

# Initialize learnable weights for the three loss components
# Start with equal weights
loss_weights = nn.Parameter(torch.ones(2, device=device) / 2)  

#optimizers 
autoencoder_optimizer = optim.AdamW(autoencoder.parameters(), lr=0.001)
predictor_optimizer = optim.AdamW(predictor.parameters(), lr=0.001)
optimizer_weights = optim.AdamW([loss_weights], lr=0.001)

num_epochs = 500

for epoch in range(num_epochs):
   
    # Autoencoder (unsupervised loss)
    encoded, reconstructed = autoencoder(input_data)
    autoencoder_loss = criterion_mse(reconstructed, input_data)
    
    #Predictor labeled (supervised loss)
    predictions = predictor(x_labeled)
    predictor_loss = criterion_CE(predictions, y_labeled)
        
    # Compute the weighted sum of the loss terms
    loss_weights_normalized = F.softmax(loss_weights, dim=0)
    total_loss = (loss_weights_normalized[0] * autoencoder_loss +
                        loss_weights_normalized[1] * predictor_loss)
                
    # Backpropagation and optimization
    autoencoder_optimizer.zero_grad()
    predictor_optimizer.zero_grad()
    optimizer_weights.zero_grad()
        
    total_loss.backward()

    autoencoder_optimizer.step()
    predictor_optimizer.step()
    optimizer_weights.step()
            
    if ((epoch % 50) == 0):
        print(f'Epoch [{epoch}/{num_epochs}], Total Loss: {total_loss.item():.3f}, Autoencoder Loss: {autoencoder_loss.item():.3f}, Predictor Loss: {predictor_loss:.3f}')
        print (f'Autoencoder Weights: {loss_weights_normalized[0].item():.3f}, Predictor Weights: {loss_weights_normalized[1].item():.3f}')

In [ ]:
def find_new_label(all_predictions, x_unlabeled, confidence, input_size, y_unlabeled, known_celltypes, y_labeled, x_labeled, print = False, apply_confidence = False):

    rows_to_remove = []
    gt_pseudo_labeled = []
    should_have_labeled = 0

    pseudo_labeled_x, pseudo_labeled_y = initialize_pseudo_labeled(input_size)
    
    for index in range(x_unlabeled.shape[0]):
        col_indices = np.where(all_predictions[index] == 1)[0]

        if len(col_indices) == 0:
            continue
                
        if len(col_indices) == 1:
            pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, should_have_labeled = \
                handle_single_class_sample(index, x_unlabeled, col_indices, pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, y_unlabeled, known_celltypes, should_have_labeled)
        elif len(col_indices) > 1:
            pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, should_have_labeled = \
                handle_multi_class_sample(index, x_unlabeled, col_indices, pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, y_unlabeled, known_celltypes, y_labeled, x_labeled, confidence, should_have_labeled, apply_confidence)
    
    if (print):
        print_results(rows_to_remove, sampled_indices, should_have_labeled)

    return rows_to_remove, gt_pseudo_labeled, pseudo_labeled_x, pseudo_labeled_y

def initialize_pseudo_labeled(input_size):
    pseudo_labeled_x = torch.empty((0, input_size), dtype=torch.float32).to(device)
    pseudo_labeled_y = torch.empty((0, 1), dtype=torch.long).to(device)
    return pseudo_labeled_x, pseudo_labeled_y

def handle_single_class_sample(index, x_unlabeled, col_indices, pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, y_unlabeled, known_celltypes, should_have_labeled):
    new_labeled_sample = np.expand_dims(x_unlabeled[index].cpu().detach().numpy(), axis=0)
    new_label = col_indices.reshape(1, 1)

    rows_to_remove.append(index)
    pseudo_labeled_x = torch.cat((pseudo_labeled_x, torch.from_numpy(new_labeled_sample).to(device)), 0)
    pseudo_labeled_y = torch.cat((pseudo_labeled_y, torch.from_numpy(new_label).to(device)), 0)
    gt_pseudo_labeled.append(y_unlabeled[index])
    
    if y_unlabeled[index].cpu().detach().numpy() in np.array(known_celltypes):
        should_have_labeled += 1
    
    return pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, should_have_labeled

def handle_multi_class_sample(index, x_unlabeled, col_indices, pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, y_unlabeled, known_celltypes, y_labeled, x_labeled, confidence, should_have_labeled, apply_confidence):
    selector = np.isin(y_labeled.cpu().detach().numpy(), col_indices)
    tmp_x = x_labeled[selector].cpu().detach().numpy()
    tmp_y = y_labeled[selector].cpu().detach().numpy()

    n_neighbors = 20
    nearNei = NearestNeighbors(n_neighbors=n_neighbors).fit(tmp_x)
    distances, indices = nearNei.kneighbors(x_unlabeled[index].cpu().detach().numpy().reshape(1, -1))
    nearest_labels = tmp_y[indices[0]]

    label_counter = Counter(nearest_labels)
    most_common_label, count = label_counter.most_common(1)[0]

    if (apply_confidence):
        if count > (n_neighbors / 2) + confidence:
            new_labeled_sample = np.expand_dims(x_unlabeled[index].cpu().detach().numpy(), axis=0)
            new_label = np.asarray(most_common_label).reshape(1, 1)

            rows_to_remove.append(index)
            pseudo_labeled_x = torch.cat((pseudo_labeled_x, torch.from_numpy(new_labeled_sample).to(device)), 0)
            pseudo_labeled_y = torch.cat((pseudo_labeled_y, torch.from_numpy(new_label).to(device)), 0)
            gt_pseudo_labeled.append(y_unlabeled[index])

            if y_unlabeled[index].cpu().detach().numpy() in np.array(known_celltypes):
                should_have_labeled += 1
    else:
        new_labeled_sample = np.expand_dims(x_unlabeled[index].cpu().detach().numpy(), axis=0)
        new_label = np.asarray(most_common_label).reshape(1, 1)

        rows_to_remove.append(index)
        pseudo_labeled_x = torch.cat((pseudo_labeled_x, torch.from_numpy(new_labeled_sample).to(device)), 0)
        pseudo_labeled_y = torch.cat((pseudo_labeled_y, torch.from_numpy(new_label).to(device)), 0)
        gt_pseudo_labeled.append(y_unlabeled[index])

        if y_unlabeled[index].cpu().detach().numpy() in np.array(known_celltypes):
            should_have_labeled += 1
    
    return pseudo_labeled_x, pseudo_labeled_y, rows_to_remove, gt_pseudo_labeled, should_have_labeled

def print_results(rows_to_remove, sampled_indices, should_have_labeled):
    
    total_labeled_samples = len(rows_to_remove)
    print('Total labeled samples is %d' % total_labeled_samples)
    print('Should have labeled %d from known classes' % len(sampled_indices))
    print('Actually labeled from known classes %d' % should_have_labeled)
    print('Known samples missing %d' % (len(sampled_indices) - should_have_labeled))
    print('Wrongly labeled %d' % (total_labeled_samples - should_have_labeled))

In [ ]:
def one_class_svm(train_encoded, test_encoded, nu, x_unlabeled, y_labeled, known_celltypes, confidence = 5, print = False, apply_confidence = False):

    all_predictions = []

    #train one-class SVM for each known cell type and perform testing on sampled unknown cells
    for k_class in np.unique(known_celltypes):
                
        x_train = train_encoded[y_labeled == k_class]
                
        ## OC-SVM MODEL
        SVM_model = OneClassSVM(kernel='rbf', gamma='auto', nu=nu, shrinking=True)
        SVM_model.fit(x_train.cpu().detach().numpy())
        predictions = SVM_model.predict(test_encoded.cpu().detach().numpy())
        all_predictions.append(predictions)

    all_predictions = np.array(all_predictions).squeeze()
    all_predictions = all_predictions.T

    #Find new labels
    rows_to_remove, gt_pseudo_labeled, pseudo_labeled_x, pseudo_labeled_y = find_new_label(all_predictions, x_unlabeled, confidence, input_size, y_unlabeled, known_celltypes, y_labeled, x_labeled, print, apply_confidence)
    return rows_to_remove, gt_pseudo_labeled, pseudo_labeled_x, pseudo_labeled_y


In [ ]:
confidence = 5
nus = [0.0001, 0.001, 0.01, 0.1, 0.3]
best_acc = 0

for nu in nus:
    print ("Running with nu = ", nu)
    rows_to_remove, gt_pseudo_labeled, pseudo_labeled_x, pseudo_labeled_y = one_class_svm(autoencoder(x_labeled)[0], autoencoder(x_unlabeled)[0], nu, x_unlabeled, y_labeled, known_celltypes, 
                                                                                          confidence, print = True, apply_confidence = False)
    
    correct = (torch.squeeze(pseudo_labeled_y) == torch.tensor(gt_pseudo_labeled)).float().sum()
    known_acc = correct / pseudo_labeled_y.shape[0]
    print(f'Supervised Accuracy: {known_acc.item()}')

    if (best_acc < known_acc.item()):
        best_acc = known_acc.item()
        best_nu = nu

    print ('-----------------------------------')
    

In [ ]:
best_nu = 0.0001
rows_to_remove, gt_pseudo_labeled, pseudo_labeled_x, pseudo_labeled_y = one_class_svm(autoencoder(x_labeled)[0], autoencoder(x_unlabeled)[0], best_nu, x_unlabeled, y_labeled, known_celltypes, 
                                                                                      confidence, print = True, apply_confidence = False)
correct = (torch.squeeze(pseudo_labeled_y) == torch.tensor(gt_pseudo_labeled)).float().sum()
known_acc = correct / pseudo_labeled_y.shape[0]
print(f'Supervised Accuracy: {known_acc.item()}')

In [ ]:
#remove the rows from the unknown data
mask = torch.ones(x_unlabeled.size(0), dtype=torch.bool)
mask[rows_to_remove] = 0
x_unlabeled = x_unlabeled[mask]
y_unlabeled = y_unlabeled[mask]
unknown_cell_names = np.delete(unknown_cell_names, rows_to_remove, axis=0)

In [ ]:
tmp = torch.squeeze(pseudo_labeled_y).cpu().detach().numpy()
tmp2 = torch.tensor(gt_pseudo_labeled, device = 'cpu').numpy()

subselected_pseudo_y = []
subselected_gt = []

for i in range(len(tmp)):
    if (tmp[i] in known_celltypes and tmp2[i] in known_celltypes):
        subselected_pseudo_y.append(tmp[i])
        subselected_gt.append(tmp2[i])

con = confusion_matrix(subselected_gt, subselected_pseudo_y)
cm_normalized = con.astype('float') / con.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues', xticklabels=known_mapping.keys(), 
             yticklabels=known_mapping.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def map_tensor(tensor, mapping):
    # Vectorize the dictionary lookup function
    vectorized_map = np.vectorize(lambda x: mapping.get(x, x))
    return vectorized_map(tensor)

all_labeled_x = torch.cat((x_labeled, pseudo_labeled_x))
all_labeled_y = torch.cat((y_labeled, torch.squeeze(pseudo_labeled_y)))

# Map the integer labels back to their original string values
string_labels = map_tensor(all_labeled_y, inv_known_mapping)

# Rember which ones are labeled and which ones are pseudo-labeled
labeled_pseudo = np.concatenate([np.zeros(x_labeled.shape[0]), np.ones(pseudo_labeled_x.shape[0])])

# Look at the wrongly pseudo labeled
comparison_array = torch.where(torch.squeeze(pseudo_labeled_y) == torch.tensor(gt_pseudo_labeled), 1, 2)
comparison_array_np = comparison_array.numpy()
wrongly_labeled = np.concatenate([np.zeros(x_labeled.shape[0]), comparison_array_np])

# Look at the samples that should have not been pseudo-labeled
should_have_not_been_labeled = np.array([1 if elem in known_celltypes else 2 for elem in torch.tensor(gt_pseudo_labeled).tolist()])
should_have_not_been_labeled = np.concatenate([np.zeros(x_labeled.shape[0]), should_have_not_been_labeled])

In [ ]:
autoencoder= autoencoder.to(device)
encoded_data = autoencoder(all_labeled_x)[0].cpu().detach().numpy()
mapper = umap.UMAP(n_neighbors=15, min_dist=0.3)
embedding = mapper.fit_transform(encoded_data)

In [ ]:
pca_umap = mapper.fit_transform(all_labeled_x)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(20,30))
ax[0,0].scatter(embedding[:, 0], embedding[:, 1], s=0.5, c=[(0.8, 0.8, 0.8) for x in string_labels])
ax[0,1].scatter(embedding[:, 0], embedding[:, 1], s=0.5, c=[color_map[x] for x in string_labels])
ax[0,1].set_xlabel('UMAP 1')
ax[0,1].set_ylabel('UMAP 2')
ax[0,1].set_yticklabels([])
ax[0,1].set_xticklabels([])
colors = [color_map[celltype] if pseudo == 1 else (0.8, 0.8, 0.8)for celltype, pseudo in zip(string_labels, labeled_pseudo)]
ax[1,0].scatter(embedding[:, 0], embedding[:, 1], s=1, c=colors)
colors = [color_map[celltype] if pseudo == 0 else (0.8, 0.8, 0.8)for celltype, pseudo in zip(string_labels, labeled_pseudo)]
ax[1,1].scatter(embedding[:, 0], embedding[:, 1], s=1, c=colors)
colors = [(0.8, 0.8, 0.8) if labeled == 0 else (0, 1, 0) if labeled == 1 else (1, 0, 0) for labeled in wrongly_labeled]
ax[2,0].scatter(embedding[:, 0], embedding[:, 1], s=1, c=colors)
colors = [(0.8, 0.8, 0.8) if labeled == 0 else (0, 1, 0) if labeled == 1 else (1, 0, 0) for labeled in should_have_not_been_labeled]
ax[2,1].scatter(embedding[:, 0], embedding[:, 1], s=1, c=colors)
#ax[2,2].scatter(pca_umap[:, 0], pca_umap[:, 1], s=0.5, c=[color_map[x] for x in string_labels])


df = pd.DataFrame(dict(ax0=embedding[:, 0], ax1=embedding[:, 1], celltype=string_labels))
mean_values = df.groupby('celltype').mean()
# mean_values["ax1"].iloc[2] = mean_values["ax1"].iloc[2] - 1
# mean_values["ax1"].iloc[1] = mean_values["ax1"].iloc[1] + 0.5
# mean_values["ax0"].iloc[9] = mean_values["ax0"].iloc[9] - 1
# mean_values["ax1"].iloc[9] = mean_values["ax1"].iloc[9] + 1
# mean_values["ax1"].iloc[3] = mean_values["ax1"].iloc[3] + 1.5
# mean_values["ax0"].iloc[4] = mean_values["ax0"].iloc[4] + 1
# mean_values["ax1"].iloc[7] = mean_values["ax1"].iloc[7] - 1

for name, row in mean_values.iterrows():
    ax[0,1].text(row.ax0, row.ax1, name, va="center", ha="center", fontdict={"weight": "bold"},fontsize=12)
    
ax[0,0].set_title('Labeled and pseudo labeled')
ax[0,1].set_title('Labeled and pseudo labeled')
ax[1,0].set_title('Labeled and pseudo labeled - Pseudo labeled')
ax[1,1].set_title('Labeled and pseudo labeled - Labeled')
ax[2,0].set_title('Wrongly pseudo labeled')
ax[2,1].set_title('Should have not been pseudo labeled')


In [ ]:
n = int(np.sqrt(len(np.unique(string_labels)))) + 1

fig, ax = plt.subplots(n, n, figsize=(40,40))

for name, axs in zip(np.unique(string_labels), ax.ravel()):
    
    row = mean_values[mean_values.index == name]

    axs.scatter(embedding[:, 0], embedding[:, 1], s=1, c=[color_map[x] if x == name else (0.8, 0.8, 0.8) for x in string_labels])
    axs.text(row.ax0.to_numpy(), row.ax1.to_numpy(), name, va="center", ha="center", fontdict={"weight": "bold"})
    axs.axis("off")
    axs.set_title(name)

### Unsupervised

In [ ]:
print ('Number of unlabeled samples', x_unlabeled.shape[0])
print ('Unkown classes: %d' % len(unknown_celltypes))
np.unique(y_unlabeled.cpu().detach().numpy(), return_counts=True)
assert (len(unknown_cell_names) == x_unlabeled.shape[0])

In [ ]:
def create_adjacency_matrix(data, num_neighbors = 50):

    num_samples = data.shape[0]
    print ('Computing adjacency using %d neighbors' %num_neighbors)
    # Number of neighbors to retrieve
    num_neighbors = num_neighbors

    # Create a cKDTree from the features matrix
    kdtree = cKDTree(data)

    # List to store nearest neighbors for each point
    nearest_neighbors_list = []

    # Query for the nearest neighbors for each point in features_matrix
    for i in range(num_samples):
        query_point = data[i]
        distances, indices = kdtree.query(query_point, k=num_neighbors)
        nearest_neighbors_list.append(indices)

    adjacency_matrix = np.zeros((num_samples, num_samples))

    # Populate the adjacency matrix based on nearest neighbors list
    for i in range(num_samples):
        nearest_neighbors_indices = nearest_neighbors_list[i]
        adjacency_matrix[i, nearest_neighbors_indices] = 1

    return adjacency_matrix


In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, n_hidden, n_layers, dropout):
        super(GCN, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(GCNConv(in_feats, n_hidden))

        # hidden layers
        for _ in range(n_layers - 1):
            self.layers.append(GCNConv(n_hidden, n_hidden))

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        for i, layer in enumerate(self.layers):
            if i != 0:
                x = self.dropout(x)
            x = F.relu(layer(x, edge_index))
        return x

class Encoder(nn.Module):
    def __init__(self, in_feats, n_hidden, n_layers, dropout):
        super(Encoder, self).__init__()
        self.conv = GCN(in_feats, n_hidden, n_layers, dropout)

    def forward(self, data, corrupt=False):
        x, edge_index = data.x, data.edge_index
        if corrupt:
            perm = torch.randperm(x.size(0))
            x = x[perm]
        x = self.conv(x, edge_index)
        return x

class Discriminator(nn.Module):
    def __init__(self, n_hidden):
        super(Discriminator, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
        self.reset_parameters()

    def uniform(self, size, tensor):
        bound = 1.0 / torch.sqrt(torch.tensor(size, dtype=torch.float))
        if tensor is not None:
            tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
        size = self.weight.size(0)
        self.uniform(size, self.weight)

    def forward(self, features, summary):
        features = torch.matmul(features, torch.matmul(self.weight, summary))
        return features

class DGI(nn.Module):
    def __init__(self, in_feats, n_hidden, n_layers, dropout):
        super(DGI, self).__init__()
        self.encoder = Encoder(in_feats, n_hidden, n_layers, dropout)
        self.discriminator = Discriminator(n_hidden)
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, data):
        positive = self.encoder(data, corrupt=False)
        negative = self.encoder(data, corrupt=True)
        summary = torch.sigmoid(positive.mean(dim=0))

        positive = self.discriminator(positive, summary)
        negative = self.discriminator(negative, summary)

        l1 = self.loss(positive, torch.ones_like(positive))
        l2 = self.loss(negative, torch.zeros_like(negative))

        return l1 + l2

In [ ]:
useGraph = True

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print ('Using ', device)

if (useGraph):
    print ('Embedding unknown cell before clustering with GNNs...')
    adj_matrix = create_adjacency_matrix(x_unlabeled)
    adj_matrix = torch.tensor(adj_matrix, dtype=torch.long)
    edge_index = adj_matrix.nonzero().t().contiguous()

    x_feat = x_unlabeled
    data = Data(x=x_feat, edge_index=edge_index)
    data = data.to(device)

    in_feats = x_unlabeled.shape[1]
    n_hidden = 32
    n_layers = 2
    dropout = 0.5
    num_epochs = 500

    model = DGI(in_feats, n_hidden, n_layers, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        optimizer.zero_grad()
        loss = model(data)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if ((epoch % 100) == 0):
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {total_loss:.4f}')

    # After training, get node embeddings
    model.eval()
    with torch.no_grad():
        node_embeddings = model.encoder(data, corrupt=False)

else: 
    node_embeddings = x_unlabeled

In [ ]:
def cluster_acc(y_pred, y_true):
    """
    Calculate clustering accuracy.
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)

    return w[row_ind, col_ind].sum() / y_pred.size

In [ ]:
component_upper_bound = 14
node_embeddings = node_embeddings.detach().numpy()
print ('Running unsupervised clustering with %d components' %component_upper_bound)
DPGMM = mixture.BayesianGaussianMixture(n_components=component_upper_bound, 
                                                max_iter=50,
                                                n_init=20,
                                                tol=1e-5, 
                                                init_params='k-means++', 
                                                weight_concentration_prior_type='dirichlet_process', verbose=0)
DPGMM.fit(node_embeddings)
print ('Unsupervised clustering done')

In [ ]:
weights = DPGMM.weights_
threshold = 0.05

# Count the number of significant components (weights above threshold)
significant_components = np.sum(weights >= threshold)
print ('Significant components: %d' %significant_components)
print ('Number of true classes: %d' %len(unknown_celltypes))

In [ ]:
DPGMM = mixture.BayesianGaussianMixture(n_components=significant_components, covariance_type = 'full',
                                                max_iter=50,
                                                n_init=50,
                                                tol=1e-5, 
                                                init_params='k-means++', 
                                                weight_concentration_prior_type='dirichlet_process', verbose=0)
DPGMM.fit(node_embeddings)

In [ ]:
clusts = pd.DataFrame(DPGMM.predict(node_embeddings))
cluster_probabilities = DPGMM.predict_proba(node_embeddings)

unknown_acc = cluster_acc(np.squeeze(clusts.to_numpy()), y_unlabeled.numpy())
print ('Unupervised Accuracy', unknown_acc)

### Final refinement

In [ ]:
tmp = clusts + len(known_celltypes)
all_labeled_x = torch.cat((pseudo_labeled_x, x_unlabeled))
all_labeled_y = torch.cat((torch.squeeze(pseudo_labeled_y), torch.tensor(np.squeeze(tmp.to_numpy()))))

# +1 because the sample itself is included
neighbors = 10
nnn = NearestNeighbors(n_neighbors=neighbors + 1)  
nnn.fit(all_labeled_x)
distances, indices = nnn.kneighbors(all_labeled_x)

# Determine the most common cluster among the neighbors
# Exclude the first neighbor because it is the sample itself

neighbor_clusters = all_labeled_y[indices[:, 1:]]  # This slices off the first column
new_clusters = np.array([mode(neighbor_clusters[i])[0][0] for i in range(neighbor_clusters.shape[0])])
print ('Superised & Unupervised Accuracy', cluster_acc(all_labeled_y.numpy(), new_clusters))

In [ ]:
unsupervised_clusters = tmp[0].to_numpy()
mapper = umap.UMAP(n_neighbors=15, min_dist=0.3)
embedding = mapper.fit_transform(node_embeddings)

# Look at the samples that should have been pseudo-labeled
should_have_been_labeled =  np.array([1 if elem in known_celltypes else 0 for elem in y_unlabeled.numpy()])

fig, ax = plt.subplots(2, 2, figsize=(20,20))
ax[0,0].scatter(embedding[:, 0], embedding[:, 1], s=1, c=[(0.8, 0.8, 0.8) for x in unsupervised_clusters])
ax[0,1].scatter(embedding[:, 0], embedding[:, 1], s=1, c=[color_map[x] for x in unsupervised_clusters])
ax[1,0].scatter(embedding[:, 0], embedding[:, 1], s=1, c=[color_map[x] for x in map_tensor(y_unlabeled, inv_all_mapping)])
colors = [(1, 0, 0) if labeled == 1 else (0, 1, 0) for labeled in should_have_been_labeled]
ax[1,1].scatter(embedding[:, 0], embedding[:, 1], s=1, c=colors)

df = pd.DataFrame(dict(ax0=embedding[:, 0], ax1=embedding[:, 1], celltype=unsupervised_clusters))
mean_values = df.groupby('celltype').mean()

for name, row in mean_values.iterrows():
    ax[0,1].text(row.ax0, row.ax1, name, va="center", ha="center", fontdict={"weight": "bold"},fontsize=12)

df = pd.DataFrame(dict(ax0=embedding[:, 0], ax1=embedding[:, 1], celltype=map_tensor(y_unlabeled, inv_all_mapping)))
mean_values = df.groupby('celltype').mean()

for name, row in mean_values.iterrows():
    ax[1,0].text(row.ax0, row.ax1, name, va="center", ha="center", fontdict={"weight": "bold"},fontsize=10)

ax[0,0].set_title('Unlabeled clustering')
ax[0,1].set_title('Unlabeled clustering')
ax[1,0].set_title('Unlabeled with ground truth')
ax[1,1].set_title('Should have pseudo labeled')

### Differential Expression on unlabeled data

In [ ]:
print ("Reading normalized unlabeled data for differential expression")
files = [f for f in listdir(join(data_dir, dataset)) if isfile(join(data_dir, dataset, f))]

for file in files:
    if ('EXPR' in file):
        exp = pd.read_csv(join(data_dir, dataset, file))
        exp = exp.T
        exp.columns = exp.iloc[0]
        exp = exp.drop(exp.index[0])
        exp = exp.apply(pd.to_numeric, errors='coerce')

exp = exp.loc[unknown_cell_names]
unsupervised_clusters = unsupervised_clusters.astype(int)

In [ ]:
# Create an AnnData object
adata = AnnData(X=exp.values, obs={'cluster': unsupervised_clusters}, var=pd.DataFrame(index=exp.columns))
adata.obs['cluster'] = adata.obs['cluster'].astype(str).astype('category')
print(adata)
sc.tl.rank_genes_groups(adata, 'cluster', method='wilcoxon', n_genes=20)
sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby="cluster", show_gene_labels=True)
sc.pl.rank_genes_groups_dotplot(adata, n_genes=5, groupby="cluster")
sc.pl.rank_genes_groups_matrixplot(adata, n_genes=5, groupby="cluster")

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

# Extract the differentially expressed genes and their metrics
de_genes = pd.DataFrame(
    {group: result['names'][group] for group in groups}
)
scores = pd.DataFrame(
    {group: result['scores'][group] for group in groups}
)
logfoldchanges = pd.DataFrame(
    {group: result['logfoldchanges'][group] for group in groups}
)
pvals = pd.DataFrame(
    {group: result['pvals'][group] for group in groups}
)
pvals_adj = pd.DataFrame(
    {group: result['pvals_adj'][group] for group in groups}
)

In [ ]:
import decoupler as dc
# Query Omnipath and get PanglaoDB
markers = dc.get_resource('PanglaoDB')
markers = markers[markers['human'] & markers['canonical_marker'] & (markers['human_sensitivity'] > 0.5)]
# Remove duplicated entries
markers = markers[~markers.duplicated(['cell_type', 'genesymbol'])]

In [ ]:
dc.run_ora(
    mat=adata,
    net=markers,
    source='cell_type',
    target='genesymbol',
    min_n=3,
    verbose=True,
    use_raw=False
)

In [ ]:
acts = dc.get_acts(adata, obsm_key='ora_estimate')
# We need to remove inf and set them to the maximum value observed for pvals=0
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e
df = dc.rank_sources_groups(acts, groupby='cluster', reference='rest', method='t-test_overestim_var')
n_ctypes = 3
ctypes_dict = df.groupby('group').head(n_ctypes).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
ctypes_dict

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

# Extract the differentially expressed genes
de_genes = pd.DataFrame(
    {group: result['names'][group] for group in groups}
)

# Print the top 20 genes for each cluster in the desired format
for group in groups:
    top_genes = de_genes[group].head(20).tolist()  # Get the top 20 genes
    genes_str = ", ".join(top_genes)
    print(f"{group}: {genes_str}")

print ('Unknow types:', np.array(unknown_celltypes_names))